# Day 8

[Day 8 description](https://adventofcode.com/2018/day/8)

I'm pretty satisfied of this solution. It's not particulary smart, but it uses pattern matching and recursion; for each "subtree" we have `n_nodes, n_metadata, *tail` where tail is another subtree, so we can use the same function recursively. The result is kind of clean. The only thing that I don't like is the `global all_metadata`, that I need to collect data in every step of the recursion.

For the second part, we need to modify the function in order to keep track of the "path" of the subtree, so we can compute the correct value for the tree itself. We only need to modify `all_metadata`, which is now a dictionary, so instead of appending metadata to the same list, we compute the key as the path to subtree (`1` is the first subtree, `2` is the second subtree; so `121` is the first subtree of the second subtree of the root node `1`) and we add it to the dict with value the list of metadata.

In [1]:
with open('AOC2018_08_input.txt') as f:
    tree = [int(x) for x in f.read().split(" ")]

In [2]:
all_metadata = list()

def find_next_nodes(seq):
    global all_metadata
    n_nodes, n_metadata, *tail = seq
    if n_nodes == 0:
        metadata, rest = tail[:n_metadata], tail[n_metadata:]
        all_metadata += metadata
    else:
        for i in range(n_nodes):
            tail = find_next_nodes(tail)            
        metadata, rest = tail[:n_metadata], tail[n_metadata:]
        all_metadata += metadata
    return rest

find_next_nodes(tree)
sum(all_metadata)

35911

In [3]:
all_metadata = dict()

def find_next_nodes_(seq, parent_name='1'):
    global all_metadata
    n_nodes, n_metadata, *tail = seq
    if n_nodes == 0:
        metadata, rest = tail[:n_metadata], tail[n_metadata:]
        all_metadata[parent_name+'-'] = sum(metadata)
    else:
        for i in range(n_nodes):
            tail = find_next_nodes_(tail, parent_name + str(i+1))
        metadata, rest = tail[:n_metadata], tail[n_metadata:]
        all_metadata[parent_name] = metadata
    return rest

def get_value_for_node(node_name, metadata):
    if node_name + '-' in metadata:
        return metadata[node_name + '-']
    elif node_name in metadata:
        node_metadata = metadata[node_name]
        subvalues = []
        for subnode in node_metadata:
            subnode_name = node_name + str(subnode)
            subvalues.append(get_value_for_node(subnode_name, all_metadata))
        return sum(subvalues)
    else:
        return 0

find_next_nodes_(tree)
get_value_for_node('1', all_metadata)

17206

In [4]:
# We can retrieve the previous result:
subtree_values = sum([v for k, v in all_metadata.items() if k[-1] == '-'])
leaves_values = sum([sum(v) for k, v in all_metadata.items() if k[-1] != '-'])
subtree_values + leaves_values

35911